In [123]:
!pip install hazm
!pip install pandas
!pip install keras
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [142]:
import hazm
import pandas as pd
import csv
from hazm import Normalizer, word_tokenize, stopwords_list, Stemmer
from hazm import Stemmer, Lemmatizer
import string
from hazm import word_tokenize
from string import punctuation
from sklearn.metrics import precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split

In [143]:
df = pd.read_csv('train.csv')
df.head(20)

,Unnamed: 0,Text,Category
0,0,\nخبرنامه دانشگاه علم و صنعت ايران \nشماره ياز...,Science and Culture
1,1,\nتا پايان سال 1378 دهها زمين فوتبال و \nسالن ...,Sport
2,2,\nانجمن توليدكنندگان تجهيزات صنعت نفت تشكيل شد...,Economy
3,3,\nكرتين براي سومين بار نخست وزير كانادا \nشد \...,Miscellaneous.World News
4,4,\nخداحافظ رفقا \nنمايندگان اروپاي شرقي در جام ...,Sport
5,5,\nبراي مقابله با فرسايش شديد خاك در \nبزرگترين...,Miscellaneous.Urban
6,6,\nنيمي از مردم جهان تحرك كافي ندارند \nهشدار س...,Sport
7,7,\nزني بدون پسوند و پيشوند \nجايگاه زن در سينما...,Social.Women
8,8,\nسن بلوغ در ايران؟ چيست \n \nبه بهانه همايش م...,Social
9,9,\nرهبران پشتون و تاجيك افغانستان در \nانتظار ت...,Miscellaneous.World News


In [144]:
df.isnull().sum()

Unnamed: 0    0
Text          0
Category      0
dtype: int64

In [145]:
first_classes = [
    'Science and Culture', 'Sport', 'Economy', 'Miscellaneous.World News', 
    'Miscellaneous.Urban', 'Social.Women', 'Social', 'Literature and Art', 
    'Politics', 'Miscellaneous', 'Economy.Bank and Bourse', 'Politics.Iran Politics', 
    'Tourism', 'Social.Religion', 'Miscellaneous.Picture', 'Miscellaneous.Happenings', 
    'Science and Culture.Science.Book', 'Literature and Art.Art', 
    'Miscellaneous.Islamic Councils', 'Literature and Art.Art.Cinema',
    'Science and Culture.Science.Information and Communication Technology', 
    'Economy.Oil', 'Economy.Commerce', 'Natural Environment',
    'Science and Culture.Science', 'Economy.Industry', 'Economy.Agriculture', 
    'Sport.World Cup', 'Miscellaneous.Picture.Caricature', 
    'Literature and Art.Art.Music', 'Literature and Art.Art.Theater',
    'Economy.Dwelling and Construction', 
    'Science and Culture.Science.Medicine and Remedy', 'Literature and Art.Literature'
]

dic_new_classes = {
    'Sport': 'athletic',
    'Science and Culture': 'cultural-art',
    'Economy': 'economic',
    'Politics': 'politics',
    'Social': 'social',
    'Literature and Art': 'cultural-art',
    'Miscellaneous': 'other',
    'Economy.Bank and Bourse': 'economic',
    'Politics.Iran Politics': 'politics',
    'Social.Women': 'social',
    'Social.Religion': 'social',
    'Economy.Oil': 'economic',
    'Economy.Commerce': 'economic',
    'Economy.Industry': 'economic',
    'Economy.Agriculture': 'economic',
    'Economy.Dwelling and Construction': 'economic',
    'Sport.World Cup': 'athletic',
    'Science and Culture.Science.Book': 'technology',
    'Science and Culture.Science.Information and Communication Technology': 'technology',
    'Science and Culture.Science': 'technology',
    'Science and Culture.Science.Medicine and Remedy': 'technology',
    'Natural Environment': 'natural environment',
    'Miscellaneous.World News': 'other',
    'Miscellaneous.Urban': 'other',
    'Miscellaneous.Picture': 'other',
    'Miscellaneous.Happenings': 'other',
    'Miscellaneous.Islamic Councils': 'other',
    'Miscellaneous.Picture.Caricature': 'other',
    'Literature and Art.Art': 'cultural-art',
    'Literature and Art.Art.Cinema': 'cultural-art',
    'Literature and Art.Art.Music': 'cultural-art',
    'Literature and Art.Art.Theater': 'cultural-art',
    'Literature and Art.Literature': 'cultural-art',
    'Tourism': 'tourism'
}


def use_dic(first_classes):
    if first_classes in dic_new_classes:
        return dic_new_classes[first_classes]
    else:
        return 'other'




df['Category'] = df['Category'].apply(use_dic)




In [146]:
df.head(20)


,Unnamed: 0,Text,Category
0,0,\nخبرنامه دانشگاه علم و صنعت ايران \nشماره ياز...,cultural-art
1,1,\nتا پايان سال 1378 دهها زمين فوتبال و \nسالن ...,athletic
2,2,\nانجمن توليدكنندگان تجهيزات صنعت نفت تشكيل شد...,economic
3,3,\nكرتين براي سومين بار نخست وزير كانادا \nشد \...,other
4,4,\nخداحافظ رفقا \nنمايندگان اروپاي شرقي در جام ...,athletic
5,5,\nبراي مقابله با فرسايش شديد خاك در \nبزرگترين...,other
6,6,\nنيمي از مردم جهان تحرك كافي ندارند \nهشدار س...,athletic
7,7,\nزني بدون پسوند و پيشوند \nجايگاه زن در سينما...,social
8,8,\nسن بلوغ در ايران؟ چيست \n \nبه بهانه همايش م...,social
9,9,\nرهبران پشتون و تاجيك افغانستان در \nانتظار ت...,other


In [147]:
df['Category'].unique()

array(['cultural-art', 'athletic', 'economic', 'other', 'social',
       'politics', 'tourism', 'technology', 'natural environment'],
      dtype=object)

In [148]:
file_path = "short.txt"
with open(file_path, "r", encoding="utf-8") as file:
    text = file.readlines()
    prepositions = []
    for i in text :
        prepositions.append(i.replace('\n',""))


prepositions.extend([".","،",":","؟","!","0","1","2","3","4","5","6","7","8","9","\u200c","؛"])
prepositions.extend(punctuation)

def remove_prepositions(text): 
    tokens = word_tokenize(text)
    useable_words = [token for token in tokens if token not in prepositions]
    new_text = " ".join(useable_words)
    return new_text

 

df['Text'] = df['Text'].apply(remove_prepositions)


In [149]:
df.head(30)  

,Unnamed: 0,Text,Category
0,0,خبرنامه دانشگاه علم صنعت ايران شماره يازدهم خب...,cultural-art
1,1,پايان سال 1378 دهها زمين فوتبال سالن ورزش كارگ...,athletic
2,2,انجمن توليدكنندگان تجهيزات صنعت نفت تشكيل مجمع...,economic
3,3,كرتين سومين وزير كانادا ژان كرتين وزير كانادا ...,other
4,4,خداحافظ رفقا نمايندگان اروپاي شرقي جام 2002 با...,athletic
5,5,مقابله فرسايش شديد خاك بزرگترين جزيره خليج فار...,other
6,6,نيمي جهان تحرك كافي هشدار سازمان بهداشت جهاني ...,athletic
7,7,زني پسوند پيشوند جايگاه زن سينماي ايران وگوباپ...,social
8,8,سن بلوغ ايران چيست بهانه همايش مسايل كودكان نو...,social
9,9,رهبران پشتون تاجيك افغانستان انتظار تصميم لويه...,other


In [177]:
df.iloc[150000,1]

'تغييرات قيمت سهام سه شنبه 19 تير تغييرقيمت آخرين قيمت تعدادسهام معامله شركت 48 2611 40058 شهد ايران 126 16849 21550 روغن نباتي پارس 480 3750 194971 مارگارين 78 7891 57600 توليدي مهرام 152 3206 6720 ايران مرينوس 147 2940 1500 بسته پارس 248 14149 1000 كارتن پارس 170 7130 42795 كارتن مشهد 468 47288 19611 نفت بهران 343 14602 16938 پتروشيمي فارابي 230 9430 147768 پتروشيمي خارك 75 3110 11595 ملي شيمي كشاورز 280 9000 1275 كيميدارو 341 7169 1236 داروسازي امين 826 5011 27408 داروسازي لرستان 39 4040 46414 كف 200 17200 13678 كربن ايران 28 1228 24833 پارسيلون 44 3705 22722 ايران تاير 13 10234 298 صنايع لاستيكي سهند 80 1680 10945 درخشان تهران 117 8485 25350 شيشه گاز 633 5200 10000 كاشي نيلو 175 3675 1000 كاشي سعدي 78 4080 8516 توليدي كاشي تكسرام 56 5727 8995 سيمان فارس خوزستان 1869 42000 2000 سيمان سپاهان 40 2520 2664 ايرانيت 113 2378 14000 ايتالران 360 1600 500 فروسيليس ايران 387 6500 58528 مس شهيد باهنر 140 5350 6879 صنعتي آما 41 2740 4400 لوازم خانگي پارس 274 5216 2572 موتوژن 10 9580 20000 صنعت

In [151]:
from sklearn.feature_extraction.text import TfidfVectorizer
from pandas.api.types import CategoricalDtype
from sklearn.ensemble import RandomForestClassifier

In [152]:
X_train_string = df['Text'].to_list()
TF_IDF_vector = TfidfVectorizer(max_features=5000)
X_train_string = TF_IDF_vector.fit_transform(X_train_string)

In [153]:
labels = [
    "cultural-art",
    "athletic",
    "economic",
    "other",
    "social",
    "politics",
    "tourism",
    "technology",
    "natural environment",
]
label_types = CategoricalDtype(labels, ordered=True)

Y_train_string = df['Category']

Y_train_string = Y_train_string.astype(label_types)
Y_train_string = pd.get_dummies(Y_train_string).values
print(Y_train_string.shape)

(150096, 9)


In [154]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X_train_string, Y_train_string, test_size=0.15, random_state=64
)

In [155]:
R_Forest = RandomForestClassifier()
R_Forest.fit(X_train, Y_train)

RandomForestClassifier()

In [156]:
import joblib
joblib.dump(R_Forest, 'rf.pkl')
joblib.dump(TF_IDF_vector, 'tfidf.pkl')

['tfidf.pkl']

In [170]:
Y_pred = R_Forest.predict(X_test)
Y_pred[10000]

array([False, False, False, False, False,  True, False, False, False])

In [162]:
from sklearn.metrics import accuracy_score, f1_score, classification_report
arg_Y_test = Y_test.argmax(axis=1)
arg_Y_pred = Y_pred.argmax(axis=1)
accuracy = accuracy_score(arg_Y_test, arg_Y_pred)         #find accuracy
print(accuracy)

0.7568731956473462


In [163]:
cmatrix = confusion_matrix(arg_Y_test, arg_Y_pred)
print(cmatrix)

[[2158    3    4  316    2    1    0    0    0]
 [  81 2005    0   15    0    1    0    0    0]
 [ 645    5 2338  223    0   16    0    0    0]
 [1378   24   78 8690    3   97    0    0    0]
 [ 864    0    6  172  236   23    0    0    0]
 [ 815    4   30  275    0 1611    0    0    0]
 [  29    2    7   18    0    0    0    0    0]
 [ 171    0    8   34    0    1    0    1    0]
 [  60    0    1   62    0    0    0    0    2]]


In [164]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.92      0.44      0.59      2484
           1       0.98      0.95      0.97      2102
           2       0.95      0.72      0.82      3227
           3       0.89      0.85      0.87     10270
           4       0.98      0.18      0.31      1301
           5       0.92      0.59      0.72      2735
           6       0.00      0.00      0.00        56
           7       1.00      0.00      0.01       215
           8       1.00      0.02      0.03       125

   micro avg       0.91      0.71      0.80     22515
   macro avg       0.85      0.42      0.48     22515
weighted avg       0.92      0.71      0.77     22515
 samples avg       0.71      0.71      0.71     22515



C:\Users\Mahdi\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Mahdi\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
